In [52]:
import pandas as pd
import numpy as np
import openpyxl
from datetime import datetime

In [53]:
files = ['WP020204 - 20-Feb-2025 12-22-34-000 PM - MainEnergy.xls', 'WP020204 - 20-Feb-2025 12-22-34-000 PM - TODEnergy.xls']

In [54]:
serial_no = files[0].split("-")[0].strip()
serial_no

'WP020204'

In [55]:
# Read the Excel file
df = pd.read_excel(files[0])
df = df.drop(index=[0, 1]).reset_index(drop=True) #drop the first two rows

# assign values of the third row as column names
df.columns = df.iloc[0]
df = df.drop(0).reset_index(drop=True)
#df.head()

In [56]:
column_list = {"Billing Date":'billing_period',"Active(I) Total":"cumulative_import",
    "Active(E) Total":"cumulative_export","Apparent-Active(I) - type 2":"apparent_import",
    "Reactive(I)":"reactive_import","Reactive(E)":"reactive_export"}

df = df.rename(columns=column_list)
df = df[df["History"] != 0] #drop this row because we can not parse its date string (its empty)
df['billing_period'] = pd.to_datetime(df['billing_period']).dt.normalize()
df['month'] = pd.to_datetime(df['billing_period']).dt.month_name()
df['year'] = pd.to_datetime(df['billing_period']).dt.year

# select needed columns
column_list = ['History', 'billing_period', 'cumulative_import', 'cumulative_export',
      'apparent_import', 'reactive_import', 'reactive_export','month', 'year']
df = df[column_list]
df = df[df["History"] == 1]
df

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025


In [57]:
df.columns

Index(['History', 'billing_period', 'cumulative_import', 'cumulative_export',
       'apparent_import', 'reactive_import', 'reactive_export', 'month',
       'year'],
      dtype='object', name=0)

In [58]:
df2 = pd.read_excel(files[1])
df2 = df2.drop(index=[0, 1]).reset_index(drop=True) #drop the first two rows

# assign values of the third row as column names
df2.columns = df2.iloc[0]
df2 = df2.drop(0).reset_index(drop=True)
df2_columns = ['History', 'Billing Date', 'TOD/SLAB', 'Active(I) Total','Active(E) Total']
df2 = df2[df2_columns]
df2.head(3)

,History,Billing Date,TOD/SLAB,Active(I) Total,Active(E) Total
0,0,----,1,96256715.1213,25016.595
1,0,----,2,58640935.1716,1532.7879
2,0,----,3,25970470.2811,18548.5367


In [59]:
df2_columns = {'Billing Date':'billing_period', 'TOD/SLAB':'tou', 'Active(I) Total':'import',
               'Active(E) Total':'export'}
df2 = df2.rename(columns=df2_columns)
df = df[df["History"] != 0] #drop the current values. also, we cannot parse its date(empty)
df['billing_period'] = pd.to_datetime(df['billing_period']).dt.normalize()
df['month'] = pd.to_datetime(df['billing_period']).dt.month_name()
df['year'] = pd.to_datetime(df['billing_period']).dt.year
df2 = df2[df2["History"] == 1]
df2.head()

,History,billing_period,tou,import,export
3,1,2025-02-01 00:00:00,1,93887499.2142,24955.1705
4,1,2025-02-01 00:00:00,2,57194922.7297,1532.7879
5,1,2025-02-01 00:00:00,3,25204181.6104,18548.5367


In [60]:
# getting rates and and assiging them to df
df['rate1'] = df2[(df2['tou'] == 1) & (df2['History'] == 1)]['import'].values[0]
df['rate2'] = df2[(df2['tou'] == 2) & (df2['History'] == 1)]['import'].values[0]
df['rate3'] = df2[(df2['tou'] == 3) & (df2['History'] == 1)]['import'].values[0]

df['rate4'] = df2[(df2['tou'] == 1) & (df2['History'] == 1)]['export'].values[0]
df['rate5'] = df2[(df2['tou'] == 2) & (df2['History'] == 1)]['export'].values[0]
df['rate6'] = df2[(df2['tou'] == 3) & (df2['History'] == 1)]['export'].values[0]
df

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year,rate1,rate2,rate3,rate4,rate5,rate6
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025,9.388750e+07,5.719492e+07,2.520418e+07,24955.1705,1532.7879,18548.5367


In [61]:
df['serial_no'] = serial_no 
df['reading_date'] = datetime.now().replace(second=0, microsecond=0)
df['vt_ratio'] = pd.NA  
df['ct_ratio'] = pd.NA  
df['no_of_resets'] = pd.NA  
df['date_of_last_reset'] = pd.NA  
df['programing_count'] = pd.NA 
df['vt_ratio'] = pd.NA  
df['ct_ratio'] = pd.NA  
df['no_of_resets'] = pd.NA  
df['date_of_last_reset'] = pd.NA  
df['programing_count'] = pd.NA 
 

In [62]:
columns_to_assign_na = ['vt_ratio', 'ct_ratio', 'no_of_resets', 'date_of_last_reset', 'programing_count',
                        'max_dem1','max_dem1_datetime','max_dem2','max_dem2_datetime','max_dem3','max_dem3_datetime']

for column in columns_to_assign_na:
    df[column] = pd.NA

df

,History,billing_period,cumulative_import,cumulative_export,apparent_import,reactive_import,reactive_export,month,year,rate1,...,ct_ratio,no_of_resets,date_of_last_reset,programing_count,max_dem1,max_dem1_datetime,max_dem2,max_dem2_datetime,max_dem3,max_dem3_datetime
1,1,2025-02-01,176406172.578899,45036.513,187551818.269299,51925429.9622,899196.0148,February,2025,9.388750e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
